In [1]:

using SparseArrays, LinearAlgebra

#=
function incomplete_lu(A::SparseMatrixCSC{T, Int}) where T
    # Dimensions of the matrix
    n = size(A, 1)
    
    # Initialize L and U as sparse matrices
    L = spdiagm(0 => ones(T, n))
    U = copy(A)
    
    # Perform the ILU factorization
    for r = 1:n-1
        # We only consider the lower triangular part for L
        for i = r+1:n
            if U[i, k] != 0
                U[i, k] /= U[k, k]
                L[i, k] = U[i, k]
                
                # We only consider the upper triangular part for U
                for j = k+1:n
                    if U[i, j] != 0
                        U[i, j] -= L[i, k] * U[k, j]
                    end
                end
            end
        end
    end
    
    # Set the strictly upper triangular part of L to zero and lower part of U
    for i = 1:n
        for j = i+1:n
            L[i, j] = 0
        end
        for j = 1:i-1
            U[i, j] = 0
        end
    end
    
    return L, U
end
=#

In [2]:
#=
# Example Usage
A = sparse([1 2 3; 4 5 6; 7 8 10])
L, U = incomplete_lu(A)

println("L: ")
println(L)
println("U: ")
println(U)'''

=#

In [21]:
using LinearAlgebra, SparseArrays
function incomplete_lu_with_sparsity(AIn)
    n = size(AIn, 1)  # Assuming A is square
    A = copy(AIn)
    for r = 1:(n-1)
        d = 1 / A[r, r]
        for i = (r+1):n
            if A[i, r] != 0
                e = d * A[i, r]
                A[i, r] = e
                for j = (r+1):n
                    if ((A[i, j] != 0)  && A[r, j] != 0)
                        A[i, j] = A[i, j] - e * A[r, j]
                    end
                end
            end
        end
    end
    return tril(A,-1)+I, triu(A,0)
end

incomplete_lu_with_sparsity (generic function with 1 method)

In [46]:
AIn = [
    4.0 2 0; 
    3 4 -2; 
    0 -1 4
    ]  # Replace with your sparse matrix

SpAIn = spzeros(3,3)
SpAIn[1,1]=SpAIn[2,2]=SpAIn[3,3]=4.0;
SpAIn[1,2]=2.0; SpAIn[2,1]=3.0;
SpAIn[2,3]=-2.0;SpAIn[3,2]=-1.0

L, U = incomplete_lu_with_sparsity(AIn)
SpL, SpU = incomplete_lu_with_sparsity(SpAIn)
#println(LU)
# error = norm(A - LU, Inf)
display(SpU)
norm(SpL-L)

3×3 SparseMatrixCSC{Float64, Int64} with 5 stored entries:
 4.0  2.0    ⋅ 
  ⋅   2.5  -2.0
  ⋅    ⋅    3.2

0.0

In [33]:
0.0 == -0

true

In [22]:

using MatrixMarket

# Replace "path_to_mtx_file.mtx" with the actual path to your .mtx file
mtx_path = "ILU0Test.mtx" 

# Read the .mtx file
B = MatrixMarket.mmread(mtx_path)

BU = incomplete_lu_with_sparsity(B)
error = norm(B - BU, Inf)

NaN

In [24]:
B

59×59 SparseMatrixCSC{Float64, Int64} with 312 stored entries:
⎡⠑⢄⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠈⠢⡀⠐⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⣠⠬⠆⠀⠑⣀⣀⡀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣿⣿⡏⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⠁⠀⠈⠐⢄⠀⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⠀⠈⠓⢄⠀⠀⠀⠑⠀⠈⢢⠤⠤⠄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⢡⣤⣤⠰⠒⠒⡄⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⣿⠀⠑⠄⡇⠀⠀⠀⎥
⎢⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⢄⠀⠀⡇⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠤⠘⢍⠉⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠱⣄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⠄⎦

In [23]:
BU

59×59 SparseMatrixCSC{Float64, Int64} with 312 stored entries:
⎡⠑⢄⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠈⠢⡀⠐⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⣠⠬⠆⠀⠑⣀⣀⡀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣿⣿⡏⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⠁⠀⠈⠐⢄⠀⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⠀⠈⠓⢄⠀⠀⠀⠑⠀⠈⢢⠤⠤⠄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⢡⣤⣤⠰⠒⠒⡄⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⣿⠀⠑⠄⡇⠀⠀⠀⎥
⎢⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⢄⠀⠀⡇⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠤⠘⢍⠉⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠱⣄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⠄⎦

In [25]:
B-BU

59×59 SparseMatrixCSC{Float64, Int64} with 2 stored entries:
⎡⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

In [27]:
norm(B - BU, Inf)

NaN

In [9]:
println(BU)

sparse([1, 44, 2, 45, 3, 46, 4, 47, 5, 48, 1, 6, 2, 7, 3, 8, 4, 9, 5, 10, 6, 12, 7, 11, 12, 13, 8, 11, 13, 14, 9, 11, 14, 15, 10, 11, 15, 16, 6, 16, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 7, 17, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 8, 18, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 9, 19, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 10, 20, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 17, 26, 55, 18, 27, 51, 56, 19, 28, 51, 57, 20, 29, 51, 58, 21, 30, 51, 59, 11, 31, 22, 32, 49, 50, 23, 33, 49, 51, 24, 34, 49, 52, 25, 35, 49, 53, 26, 36, 49, 54, 22, 33, 37, 23, 34, 38, 24, 35, 39, 25, 36, 40, 26, 37, 41, 27, 28, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 27, 29, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 27, 30, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 27, 31, 39, 40, 41